In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 10
LAYER_SIZE = 10
S_LAYER_SIZE = 15
LATENT_SIZE = 4

In [3]:
# Load the data from the file

saved = np.load("testing1.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.Size([50000, 11])


11

In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.l2a = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [7]:
def train(e_count, model, optimizer):
    data = train_data
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
train_data=train_data[torch.randperm(train_data.size()[0])]

In [10]:
vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=0.001)

In [11]:
for i in range(EPOCHS):
    train(i, vae1, opt1)

Epoch: 0, Loss: 7.157643240966797
Epoch: 1, Loss: 6.917936082763672
Epoch: 2, Loss: 6.911544809570312
Epoch: 3, Loss: 6.908651323242188
Epoch: 4, Loss: 6.907659614257812
Epoch: 5, Loss: 6.906795903320313
Epoch: 6, Loss: 6.90655442993164
Epoch: 7, Loss: 6.906230411376953
Epoch: 8, Loss: 6.905259224853515
Epoch: 9, Loss: 6.904769527587891


In [12]:
import gym
env = gym.make("InvertedPendulum-v2")
env.reset()
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0] 
action_low = env.action_space.low[0]
action_high = env.action_space.high[0]
state_low = -10.0
state_high = 10.0

In [13]:
def descale(x):
    # State
    ((x[:, 0].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 1].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 2].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 3].mul_(state_high-state_low)).add_(state_low)).to(device)

    # Action
    ((x[:, 4].mul_(action_high-action_low)).add_(action_low)).to(device)
    
    # State
    ((x[:, 5].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 6].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 7].mul_(state_high-state_low)).add_(state_low)).to(device)
    ((x[:, 8].mul_(state_high-state_low)).add_(state_low)).to(device)
    
    # Reward
    (x[:, 9].mul_(20.0)).to(device)
    
    # Done
    (x[:, 10].round_()).to(device)
    
    return x

In [14]:
from torch.autograd import Variable

print("Action")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 1).to("cuda")
    print(sample[0][5])
    x = vae1(sample)
    print(x[0][0][5], '\n')

Action
tensor(0.2121, device='cuda:0')
tensor(0.5000, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1684, device='cuda:0')
tensor(0.5000, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.3308, device='cuda:0')
tensor(0.5003, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.7878, device='cuda:0')
tensor(0.4999, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.2572, device='cuda:0')
tensor(0.4999, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.3794, device='cuda:0')
tensor(0.5000, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.9485, device='cuda:0')
tensor(0.5000, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.4474, device='cuda:0')
tensor(0.4998, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.3523, device='cuda:0')
tensor(0.5002, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.8089, device='cuda:0')
tensor(0.4994, device='cuda:0', grad_fn=<SelectBackward>) 



In [15]:
print("Reward")
for i in range(10):
    sample = torch.FloatTensor(2, 11).uniform_(0, 0.15).to("cuda")
    print(sample[0][9])
    x = vae1(sample)
    print(x[0][0][9], '\n')

Reward
tensor(0.0133, device='cuda:0')
tensor(0.0538, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0845, device='cuda:0')
tensor(0.0563, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0650, device='cuda:0')
tensor(0.0476, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1282, device='cuda:0')
tensor(0.0429, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0446, device='cuda:0')
tensor(0.0539, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.1449, device='cuda:0')
tensor(0.0500, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0027, device='cuda:0')
tensor(0.0545, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0762, device='cuda:0')
tensor(0.0599, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0399, device='cuda:0')
tensor(0.0560, device='cuda:0', grad_fn=<SelectBackward>) 

tensor(0.0854, device='cuda:0')
tensor(0.0343, device='cuda:0', grad_fn=<SelectBackward>) 



In [16]:
torch.set_printoptions(profile="short")
for i in range(5):
    sample = torch.FloatTensor(1, 11).uniform_(0, 1).to("cuda")
    x, m, s = vae1(sample)
    loss = loss_function(x, sample, m, s)
    
    x = x.to("cpu")
    sample = sample.to("cpu")
    
    print(f"Original:       {sample}")
    print(f"Reconstruction: {x}")
    print(f"Loss: {loss}\n\n")

Original:       tensor([[0.12, 0.41, 0.95, 0.82, 0.21, 0.03, 0.78, 0.19, 0.15, 0.86, 0.52]])
Reconstruction: tensor([[0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.51, 0.50, 0.03, 0.15]],
       grad_fn=<CopyBackwards>)
Loss: 10.315363883972168


Original:       tensor([[0.52, 0.42, 0.16, 0.44, 0.97, 0.59, 0.35, 0.01, 0.51, 0.27, 0.18]])
Reconstruction: tensor([[0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.05, 0.14]],
       grad_fn=<CopyBackwards>)
Loss: 7.566251754760742


Original:       tensor([[0.52, 0.01, 0.53, 0.65, 0.71, 0.73, 0.96, 0.83, 0.57, 0.76, 0.62]])
Reconstruction: tensor([[0.50, 0.50, 0.50, 0.50, 0.51, 0.50, 0.50, 0.50, 0.50, 0.05, 0.17]],
       grad_fn=<CopyBackwards>)
Loss: 9.725531578063965


Original:       tensor([[0.31, 0.10, 0.73, 0.32, 0.72, 1.00, 0.45, 0.20, 0.15, 0.79, 0.32]])
Reconstruction: tensor([[0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.05, 0.20]],
       grad_fn=<CopyBackwards>)
Loss: 9.229823112487793


Original:       tensor(